In [1]:
import numpy as np
from keras.models import load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
import random

2024-02-21 15:26:34.248845: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-21 15:26:34.352495: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 15:26:34.694223: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:
2024-02-21 15:26:34.694285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
model2 = tf.keras.models.load_model('CVE2023-v2.h5')
model1 = tf.keras.models.load_model('CVE2023.h5')
model3 = tf.keras.models.load_model('CVE2023-v3.h5')


2024-02-21 15:26:35.252511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-21 15:26:35.256036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-21 15:26:35.256148: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-21 15:26:35.256506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [63]:
#model1.summary()
#model2.summary()
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 1000)              4000      
                                                                 
 dense_12 (Dense)            (None, 500)               500500    
                                                                 
 dense_13 (Dense)            (None, 250)               125250    
                                                                 
 dense_14 (Dense)            (None, 1)                 251       
                                                                 
Total params: 630,001
Trainable params: 630,001
Non-trainable params: 0
_________________________________________________________________


In [4]:
import math
n = 4294967295
print(n/10**int(math.log10(n)+1))

0.4294967295


In [36]:
def test_overflow(input):
    #print(input[0])
    if input[0] > 4294967295:
        return 1
    else:
        return 0
def run_test(input1,input2,size):
    suss = 0
    for i in range(0,size):
        if(test_overflow(input1[i]) == test_overflow(input2[i])):
            suss = suss+ 1
        #else:
            #print(input1[i],input2[i])
    return suss
def get_int_rand(n):
    result1=[]
    result2=[]
    result3=[]
    result4=[]
    #print(n)
    for i in range(0,n):
        ratio1 = random.randint(1,10)
        ratio2 = random.randint(1,10)
        ratio3 = random.randint(1,10)
        int1 = random.randint(1,10**ratio1)
        int2 = random.randint(1,10**ratio2)
        int3 = random.randint(1,10**ratio3)
        result4 +=[[int1,int2,int3]]
        result3 +=[[int(math.log10(int1)+1) + int(math.log10(int2)+1) +int(math.log10(int3)+1)]]
        result2 +=[[int1*int2*int3]]
        int1 = int1/10**int(math.log10(int1)+1)
        int2 = int2/10**int(math.log10(int2)+1)
        int3 = int3/10**int(math.log10(int3)+1)
        result1 +=[[int1,int2,int3]]
    return np.array(result1),np.array(result2),np.array(result3),np.array(result4)

def input_num(int1,int2,int3):
    result1=[]
    result2=[]
    result3=[]
    result4=[]
    result4 +=[[int1,int2,int3]]
    result3 +=[[int(math.log10(int1)+1) + int(math.log10(int2)+1) +int(math.log10(int3)+1)]]
    result2 +=[[int1*int2*int3]]
    int1 = int1/10**int(math.log10(int1)+1)
    int2 = int2/10**int(math.log10(int2)+1)
    int3 = int3/10**int(math.log10(int3)+1)
    result1 +=[[int1,int2,int3]]
    return np.array(result1),np.array(result2),np.array(result3),np.array(result4)

In [29]:
from decimal import Decimal, getcontext
getcontext().prec = 50
def int_rand_test (constx):
    x , ans, offset, x2= get_int_rand(constx)
    
    #x , ans, offset, x2= input_num(int1,int2,int3)
    x2 = np.sort(x2, axis=-1)[:, ::-1]
    print(x2)
    print("x = ",x)
    #print(offset)
    z=[]
    y = model1.predict(x)
    y2 = model2.predict(x2)
    y3 = model3.predict(x2)
    pass_y1 = 0
    pass_y2 = 0
    pass_y3 = 0
    for i in range(0,constx):
        #z = y[i]* (10**offset[i])
        off = float(Decimal('10') **offset[i])
        z +=[[y[i]*off]]
        #print(z[i], ans[i],int(math.log10(z[i][0])),int(math.log10(ans[i])))
    for i in range(0,constx):
        if(y2[i] >= 0.2 and ans[i] >= 4294967295):
            pass_y2 += 1
        elif(y2[i] < 0.2 and ans[i] < 4294967295):
            pass_y2 += 1
    for i in range(0,constx):
        if(y3[i] >= 0.2 and ans[i] >= 4294967295):
            pass_y3 += 1
        elif(y3[i] < 0.2 and ans[i] < 4294967295):
            pass_y3 += 1
    pass_y1 = run_test(z,ans,constx)
    print("pass case of model1's number == ",pass_y1)
    print("pass case of model2's number == ",pass_y2)
    print("pass case of model3's number == ",pass_y3)
    return pass_y1,pass_y2,pass_y3


In [7]:
model1.predict(np.array([[0.1,0.1,0.1]]))

1/1 [==============================] - 1s 531ms/step


2024-02-21 15:26:37.417473: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


array([[0.00161014]], dtype=float32)

In [30]:
pass_y1, pass_y2, pass_y3 = int_rand_test(10000000)

[[9668671788  561856111        644]
 [4817614460 2680758435   34974205]
 [ 768413313   59285050      13514]
 [ 270420032      48641       4875]
 [ 326944216   32085143     181076]
 [ 220925852    5878929        402]
 [7778128593    4415167       7561]
 [  11192573         22          4]
 [ 780736020     546891         83]
 [ 148717002    5102780        816]]
x =  [[0.96686718 0.56185611 0.644     ]
 [0.34974205 0.26807584 0.48176145]
 [0.13514    0.5928505  0.76841331]
 [0.27042003 0.4875     0.48641   ]
 [0.32694422 0.181076   0.32085143]
 [0.22092585 0.5878929  0.402     ]
 [0.7561     0.4415167  0.77781286]
 [0.11192573 0.4        0.22      ]
 [0.546891   0.78073602 0.83      ]
 [0.816      0.148717   0.510278  ]]
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step
pass case of model1's number ==  10
pass case of model2's number ==  9
pass case of model3's number ==  9


/tmp/ipykernel_195903/4131390466.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  off = float(Decimal('10') **offset[i])


In [18]:


print(f"Model 1 correctness: {9997476 / 10000000 * 100}%")
print(f"Model 2 correctness: {8610601 / 10000000 * 100}%")
print(f"Model 3 correctness: {8610601 / 10000000 * 100}%")


Model 1 correctness: 99.97475999999999%
Model 2 correctness: 86.10601%
Model 3 correctness: 86.10601%


In [59]:
UINT_MAX = 4294967295.0
#normal input
input1 = np.array([[UINT_MAX-1,UINT_MAX,2147483648.0]])

#problem input
input2 = np.array([[UINT_MAX,UINT_MAX,2147483648.0]])
input3 = np.array([[UINT_MAX-100,UINT_MAX-5,2849931574.0]])
x, ans, offset, x2 = input_num(input3[0][0],input3[0][1],input3[0][2])

x2 = np.sort(x2, axis=-1)[:, ::-1]
z=[]
y = model1.predict(x)
y2 = model2.predict(x2)
y3 = model3.predict(x2)
pass_y1 = 0
pass_y2 = 0
pass_y3 = 0
#print("input = ",input2)
print("input = {:.0f}".format(input3[0][0]),"{:.0f}".format(input3[0][1]),"{:.0f}".format(input3[0][2]))
for i in range(0,1):
    #z = y[i]* (10**offset[i])
    off = float(Decimal('10') **offset[i])
    z +=[[y[i]*off]]
    print(z[i], ans[i])
    run_test(z,ans,1)
for i in range(0,1):
    if(y2[i] >= 0.5):
        print("predict overflow",end="\t")
    elif(y2[i] < 0.5 ):
        print("predict not overflow",end="\t")
    if(ans[i] >= 4294967295):
        print("real overflow")
    else:
        print("real not overflow")
for i in range(0,1):
    if(y2[i] >= 0.5):
        print("predict overflow",end="\t")
    elif(y2[i] < 0.5 ):
        print("predict not overflow",end="\t")
    if(ans[i] >= 4294967295):
        print("real overflow")
    else:
        print("real not overflow")


1/1 [==============================] - 0s 16ms/step
input = 4294967195 4294967290 2849931574
[array([5.28572e+28], dtype=float32)] [5.25719571e+28]
predict overflow	real overflow
predict overflow	real overflow


/tmp/ipykernel_195903/1574605883.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  off = float(Decimal('10') **offset[i])
